In [3]:
from gat_pred import DataBuilder

builder = DataBuilder()


In [4]:
builder.load_papers()
builder.load_methods()
builder.load_tasks()
builder.data

100%|██████████| 1417/1417 [00:07<00:00, 200.58it/s]


HeteroData(
  paper={ x=[5364, 768] },
  method={ x=[907, 768] },
  task={ x=[1417, 768] }
)

In [5]:
builder.load_relations()
builder.data

HeteroData(
  paper={ x=[5364, 768] },
  method={ x=[907, 768] },
  task={ x=[1417, 768] },
  user={ x=[700, 768] },
  (paper, cites, paper)={ edge_index=[2, 98970] },
  (paper, applies, method)={ edge_index=[2, 14916] },
  (paper, performs, task)={ edge_index=[2, 15072] },
  (user, likes, paper)={ edge_index=[2, 2334] }
)

In [6]:
import torch_geometric.transforms as T
data = T.ToUndirected()(builder.data.to("cuda"))
data

HeteroData(
  paper={ x=[5364, 768] },
  method={ x=[907, 768] },
  task={ x=[1417, 768] },
  user={ x=[700, 768] },
  (paper, cites, paper)={ edge_index=[2, 196576] },
  (paper, applies, method)={ edge_index=[2, 14916] },
  (paper, performs, task)={ edge_index=[2, 15072] },
  (user, likes, paper)={ edge_index=[2, 2334] },
  (method, rev_applies, paper)={ edge_index=[2, 14916] },
  (task, rev_performs, paper)={ edge_index=[2, 15072] },
  (paper, rev_likes, user)={ edge_index=[2, 2334] }
)

In [7]:
import torch
test_data = builder.data
test_data["user", "likes", "paper"].edge_index = torch.tensor([
    [0],
    [2946]
])
test_data["user"].x = test_data["paper"].x[2946].unsqueeze(0)
test_data = T.ToUndirected()(test_data.to("cuda"))
test_data

HeteroData(
  paper={ x=[5364, 768] },
  method={ x=[907, 768] },
  task={ x=[1417, 768] },
  user={ x=[1, 768] },
  (paper, cites, paper)={ edge_index=[2, 196576] },
  (paper, applies, method)={ edge_index=[2, 14916] },
  (paper, performs, task)={ edge_index=[2, 15072] },
  (user, likes, paper)={ edge_index=[2, 1] },
  (method, rev_applies, paper)={ edge_index=[2, 14916] },
  (task, rev_performs, paper)={ edge_index=[2, 15072] },
  (paper, rev_likes, user)={ edge_index=[2, 1] }
)

In [8]:
transform = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    disjoint_train_ratio=0.3,
    neg_sampling_ratio=2.0,
    add_negative_train_samples=False,
    edge_types=("user", "likes", "paper"),
    rev_edge_types=("paper", "rev_likes", "user"), 
)
train, val, test = transform(data)

In [17]:
from torch_geometric.loader import LinkNeighborLoader

edge_label_index = train["user", "likes", "paper"].edge_label_index
edge_label = train["user", "likes", "paper"].edge_label
train_loader = LinkNeighborLoader(
    data=train,
    num_neighbors=[10, 5],
    neg_sampling_ratio=2.0,
    edge_label_index=(("user", "likes", "paper"), edge_label_index),
    edge_label=edge_label,
    batch_size=1,
    shuffle=True,
)

In [18]:
import torch
from torch_geometric.nn import GATConv, to_hetero
from torch_geometric.data import HeteroData
class GNN(torch.nn.Module):
    def __init__(self, ):
        super().__init__()
        self.conv1 = GATConv(768, 128, heads=8, add_self_loops=False)
        self.relu = torch.nn.ReLU()
        self.conv2 = GATConv(128 * 8, 768, heads=1, add_self_loops=False)
    
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = self.relu(x)
        x = self.conv2(x, edge_index)
        return x

class GATPred(torch.nn.Module):
    def __init__(self, ):
        super().__init__()
        self.gnn = to_hetero(GNN(), metadata=data.metadata())
    
    def forward(self, x: HeteroData):
        x_dict = self.gnn({
            "user": x["user"].x,
            "paper": x["paper"].x,
            "method": x["method"].x,
            "task": x["task"].x,
        }, x.edge_index_dict)
        edge_label_index = x["likes"].edge_label_index
        user_edge_features = x_dict["user"][edge_label_index[0]]
        paper_edge_features = x_dict["paper"][edge_label_index[1]]
        return torch.sum(user_edge_features * paper_edge_features, dim=-1)
    
model = GATPred()
model = model.to("cuda")

In [19]:
from tqdm import tqdm

optim = torch.optim.SGD(model.parameters(), lr=0.001)
loss_fn = torch.nn.BCEWithLogitsLoss()

for epoch in range(50):
    total_loss = total_examples = 0
    for d in tqdm(train_loader):
        optim.zero_grad()
        pred = model(d.to("cuda"))
        gt = d["likes"].edge_label
        loss = loss_fn(pred, gt)
        loss.backward()
        optim.step()
        total_loss += float(loss) * pred.numel()
        total_examples += pred.numel()
    print(f"Epoch {epoch}: {total_loss / total_examples}")

100%|██████████| 560/560 [00:03<00:00, 170.71it/s]


Epoch 0: 2.0400003738701344


100%|██████████| 560/560 [00:03<00:00, 153.98it/s]


Epoch 1: 0.4495722593673106


100%|██████████| 560/560 [00:03<00:00, 162.61it/s]


...

Epoch 47: 0.37086351539141366


100%|██████████| 560/560 [00:03<00:00, 173.40it/s]


Epoch 48: 0.3771397073553609


100%|██████████| 560/560 [00:03<00:00, 173.28it/s]

Epoch 49: 0.4050315668606865


In [21]:
torch.save(model.state_dict(), "gat_pred.pth")

In [22]:
model.eval()
test_data["likes"].edge_label_index = test_data["likes"].edge_index
model(test_data)

tensor([1.2291], device='cuda:0', grad_fn=<SumBackward1>)